In [13]:
import $file.common
import common._
import cats._, cats.implicits._, cats.data._
import fs2.Stream
import doobie._, doobie.implicits._

import $file.$     

import common._

import cats._, cats.implicits._, cats.data._

import fs2.Stream

import doobie._, doobie.implicits._

# Variation 6. MTL Repositories

Same case classes.

In [14]:
case class Country(code: String, name: String, capital: Option[Int])
case class City(id: Int, name: String, countryCode: String, population: Int)

defined class Country
defined class City

Abstract the particular computation away.

In [15]:
trait CityAlg[F[_]]{
    def getCityName(id: Int): F[String]
    def getCityPopulation(id: Int): F[Int]
}

trait CountryAlg[F[_]]{
    def getAllCountries: F[Country]
    def getCountryName(code: String): F[String]
    def getCountryCapital(code: String): F[Int]
}

defined trait CityAlg
defined trait CountryAlg

Query builds upon domain repositories and standard algebras for imperative programming (i.e. Monad, etc.):

In [16]:
def largeCapitals[F[_]: Monad: FunctorFilter](implicit 
        Co: CountryAlg[F], Ci: CityAlg[F]): F[(String, String)] = for {
    Country(_, name, Some(capital)) <- Co.getAllCountries
    population <- Ci.getCityPopulation(capital)
    if population > 8000000
    cityName <- Ci.getCityName(capital)
} yield (cityName, name)

defined function largeCapitals

### Doobie implementation (streaming)

In [17]:
implicit object CityAlgS3 extends CityAlg[λ[T => Stream[ConnectionIO, T]]]{
    def getCityName(id: Int): Stream[ConnectionIO, String] = 
        sql"select name from city where id = $id"
            .query[String].stream
    
    def getCityPopulation(id: Int): Stream[ConnectionIO,Int] = 
        sql"select population from city where id = $id"
            .query[Int].stream
}

implicit object CountryAlgS3 extends CountryAlg[λ[T => Stream[ConnectionIO, T]]]{
    def getAllCountries: Stream[ConnectionIO, Country] = 
        sql"select code, name, capital from country"
            .query[Country].stream
    
    def getCountryName(code: String): Stream[ConnectionIO,String] =
        sql"select name from country where id = $code"
            .query[String].stream
    
    def getCountryCapital(code: String): Stream[ConnectionIO, Int] =
        sql"select capital from country where id = $code"
            .query[Int].stream
}

defined object CityAlgS3
defined object CountryAlgS3

In [18]:
largeCapitals[λ[T => Stream[ConnectionIO, T]]].compile
    .toList
    .transact(xa)
    .unsafeRunSync
    .timed

502 millis


res17: List[(String, String)] = List(
  ("Jakarta", "Indonesia"),
  ("Seoul", "South Korea"),
  ("Ciudad de M\u00e9xico", "Mexico"),
  ("Moscow", "Russian Federation")
)

### Pure model with StateT

In [19]:
case class World(countries: Map[String, Country], cities: Map[Int, City])

defined class World

In [20]:
type WorldState[T] = State[World, List[T]]

implicit object CityWorldState extends CityAlg[WorldState]{
    def getCityName(id: Int): WorldState[String] = 
        StateT(world => Eval.now(world, 
             world.cities.get(id).map(city => city.name).toList))
    
    def getCityPopulation(id: Int): WorldState[Int] = 
        StateT(world => Eval.now(world, 
             world.cities.get(id).map(city => city.population).toList))
}

implicit object CountryWorldState extends CountryAlg[WorldState]{
    def getAllCountries: WorldState[Country] = 
        StateT(world => Eval.now(world, 
            world.countries.values.toList))

    def getCountryName(code: String): WorldState[String] =
        StateT(world => Eval.now(world, 
             world.countries.get(code).map(country => country.name).toList))
    
    def getCountryCapital(code: String): WorldState[Int] =
        StateT(world => Eval.now(world, 
             world.countries.get(code).flatMap(country => country.capital).toList))
}

defined type WorldState
defined object CityWorldState
defined object CountryWorldState

In [21]:
val world: World =         
    World(Map("ES" -> Country("ES","Spain",Some(0)),
            "USA" -> Country("USA", "United States", Some(1)),
            "UK" -> Country("UK", "United Kingdom", Some(2)),
            "UNK" -> Country("UNK", "Unknown", None)),
        Map(0->City(0,"Madrid","ES",9000000),
            1->City(1,"Washington", "USA", 10000000),
            2->City(2,"London", "UK", 500000)))

world: World = World(
  Map(
    "ES" -> Country("ES", "Spain", Some(0)),
    "USA" -> Country("USA", "United States", Some(1)),
    "UK" -> Country("UK", "United Kingdom", Some(2)),
    "UNK" -> Country("UNK", "Unknown", None)
  ),
  Map(
    0 -> City(0, "Madrid", "ES", 9000000),
    1 -> City(1, "Washington", "USA", 10000000),
    2 -> City(2, "London", "UK", 500000)
  )
)

In [22]:
largeCapitals[WorldState].run(world).value

res21: (World, List[(String, String)]) = (
  World(
    Map(
      "ES" -> Country("ES", "Spain", Some(0)),
      "USA" -> Country("USA", "United States", Some(1)),
      "UK" -> Country("UK", "United Kingdom", Some(2)),
      "UNK" -> Country("UNK", "Unknown", None)
    ),
    Map(
      0 -> City(0, "Madrid", "ES", 9000000),
      1 -> City(1, "Washington", "USA", 10000000),
      2 -> City(2, "London", "UK", 500000)
    )
  ),
  List(("Madrid", "Spain"), ("Washington", "United States"))
)

In [23]:
val realCountries: List[Country] = 
    sql"select code, name, capital from country"
        .query[Country].to[List].transact(xa).unsafeRunSync

val realCities: List[City] = 
    sql"select id, name, countryCode, population from city"
        .query[City].to[List].transact(xa).unsafeRunSync

val realMapCities: Map[Int, City] = 
    Map.from(realCities.map(city => (city.id, city)))

val realWorld = World(
    Map.from(realCountries.map(c => (c.code, c))), realMapCities)

realCountries: List[Country] = List(
  Country("AFG", "Afghanistan", Some(1)),
  Country("NLD", "Netherlands", Some(5)),
  Country("ANT", "Netherlands Antilles", Some(33)),
  Country("ALB", "Albania", Some(34)),
  Country("DZA", "Algeria", Some(35)),
  Country("ASM", "American Samoa", Some(54)),
  Country("AND", "Andorra", Some(55)),
  Country("AGO", "Angola", Some(56)),
  Country("AIA", "Anguilla", Some(62)),
  Country("ATG", "Antigua and Barbuda", Some(63)),
  Country("ARE", "United Arab Emirates", Some(65)),
  Country("ARG", "Argentina", Some(69)),
  Country("ARM", "Armenia", Some(126)),
  Country("ABW", "Aruba", Some(129)),
  Country("AUS", "Australia", Some(135)),
  Country("AZE", "Azerbaijan", Some(144)),
  Country("BHS", "Bahamas", Some(148)),
  Country("BHR", "Bahrain", Some(149)),
  Country("BGD", "Bangladesh", Some(150)),
  Country("BRB", "Barbados", Some(174)),
  Country("BEL", "Belgium", Some(179)),
  Country("BLZ", "Belize", Some(185)),
  Country("BEN", "Benin", Some(187))

In [24]:
largeCapitals[WorldState]
    .runA(realWorld)
    .value
    .timed

46 millis


res23: List[(String, String)] = List(
  ("Jakarta", "Indonesia"),
  ("Ciudad de M\u00e9xico", "Mexico"),
  ("Moscow", "Russian Federation"),
  ("Seoul", "South Korea")
)